<a href="https://colab.research.google.com/github/peremartra/Large-Language-Model-Notebooks-Course/blob/main/6-PRUNING/6_6_pruning_attention_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
    <h1>Large Language Models Projects</a></h1>
    <h3>Apply and Implement Strategies for Large Language Models</h3>
    <h2>Pruning Attention Layers</h2>
    <h3>Not All Attention is needed</h3>
</div>

by [Pere Martra](https://www.linkedin.com/in/pere-martra/)

_______
Models: meta-llama/Llama-3.2

Colab Environment: GPU L4 for 3B Models

T4 for 1B Model.

Keys:
* Pruning
* Attention

References:
* [What Matters in Transformers? Not All Attention is Needed](https://arxiv.org/abs/2406.15786)
* [Resource-Efficient Transformer Pruning for Finetuning of Large Models](https://openaccess.thecvf.com/content/CVPR2024/html/Ilhan_Resource-Efficient_Transformer_Pruning_for_Finetuning_of_Large_Models_CVPR_2024_paper.html)

_______
**disclaimer: The pruning / knowledge distillation section has been created after the first edition of the book was published. They are not included in the book’s original content but are intended to supplement and expand on the topics covered.**

This is the unofficial repository for the book:
        <a href="https://amzn.to/4eanT1g"> <b>Large Language Models:</b> Apply and Implement Strategies for Large Language Models</a> (Apress).
        The book is based on the content of this repository, but the notebooks are being updated, and I am incorporating new examples and chapters.
        If you are looking for the official repository for the book, with the original notebooks, you should visit the
        <a href="https://github.com/Apress/Large-Language-Models-Projects">Apress repository</a>, where you can find all the notebooks in their original format as they appear in the book.

______
# Introduction
This notebook implements the paper: [What Matters in Transformers? Not all Attention is Needed](https://arxiv.org/abs/2406.15786).
Although I followed the paper's guidelines, I made some adjustments to make the code clearer and easier to understand.

The original paper demonstrates that larger models tend to have excessive redundancy in their attention layers. They achieved a 48.4% increase in inference performance for a Llama-2-70B model with only a minor 2.4% drop in response quality, **just bypassing the 50% of the Attention layers!**

In this notebook, tests have been conducted using Llama-3.2-1B and 3B models. With these models, I found that removing 50% of the attention layers significantly impacted the model's functionality. However, the 3B model handled the removal of these layers much better. This suggests that redundancy may become more pronounced as model size increases.

# Methodology.
To identify which layers contribute the least, the cosine distance between the layer's input and output is measured. In the paper, this distance is calculated using a test dataset, while in the notebook, I used a simple prompt to activate the layers.  

This method of measuring the importance of attention layers and their contribution to the model allows pruning to be tailored to a specific dataset. This approach can lead to the creation of more efficient models for specialized sectors such as healthcare or finance.


Once the layer contributing the least to the final output is identified (the one with the smallest difference between input and output), it is added to a list included in the configuration file.

This list is then referenced during inference by a new forward function that replaces the original one for attention layers. When this new function detects that a layer is in the list, it skips its execution and simply returns the input without modifications.

The process of identifying layers to deactivate and marking them as non-executable is one-shot. In other words, it does  determine all the layers to skip in one go, as recommended in the paper.

The iterative implementation has a significant drawback: the test dataset must be processed for each layer to be deactivated. The paper's authors note that while the iterative method may bring slight improvements, the added computational cost is not justified. However, since this is an example notebook, and there is no test dataset—just a small prompt—and the layer selection process takes only seconds, I chose the iterative approach.

This pruning method does not produce a smaller model, as the layers are not physically removed. They remain in the model but are not executed, resulting in improved inference response times.
______

# Install libraries & Configure variables.

In [ ]:
!pip install -q torch==2.6.0
!pip install -q torchvision==0.21.0
!pip install -q transformers==4.51.3
!pip install -q datasets==3.6.0
!pip install -q lm-eval==0.4.8

!pip install hf_xet #To speed up downloads from HF.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does 

In [ ]:
import logging
import math
import os
import sys
import shutil
from copy import deepcopy

import torch
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer


# Download the Model.

In [ ]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [ ]:
#model_name = 'meta-llama/Llama-3.2-1B'
model_name = 'meta-llama/Llama-3.2-3B'
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)
#tokenizer.pad_token = tokenizer.eos_token  # Set pad token

## Study the structure.
* Llama-3.2-1B
```
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb): LlamaRotaryEmbedding()
  )
  (lm_head): Linear(in_features=2048, out_features=128256, bias=False)
)
```


The model follows the typical structure of modern Llama models, consisting of blocks made up of an Attention layer and an MLP layer with a GLU structure.

> If you want to see an example of how to perform pruning on the MLP layers of the model, you can check out the notebook:[Pruning Llama 3.2.](https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/6-PRUNING/6_3_pruning_structured_llama3.2-1b_OK.ipynb) y leer el paper [Exploring GLU expansion ratios: Structured pruning in Llama-3.2 models](https://osf.io/preprints/osf/qgxea)



Since the layers form a block, the attention layer cannot be removed without also removing the accompanying MLP layer. For this reason, the decision was made to deactivate their execution during inference.

The 1B model has 16 layers, as shown in the structure above, while the 3B model has 28 layers.


# Inference function & Test Base Model

The `get_output` function is designed to generate text  and measure the time taken for different stages of the generation process.

It provides insights into the performance of the model and can be used to evaluate the efficiency of text generation.

In [ ]:
import time

def get_output(prompt, model=model, tokenizer=tokenizer, num_runs=1, max_length=50):
    total_time = 0
    generated_outputs = []

    for run in range(num_runs):
        # Start timing
        start_time = time.time()

        # Tokenization time
        token_start = time.time()
        inputs = tokenizer(prompt, return_tensors='pt').to(device)
        token_time = time.time() - token_start

        # Generation time
        gen_start = time.time()
        outputs = model.generate(
            inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length=max_length,
            num_return_sequences=1,
            pad_token_id=tokenizer.pad_token_id,
            temperature=None,
            top_p=None,
            do_sample=False,  # Disable sampling
            num_beams=5,      # Use beam search
            early_stopping=True,  # Stop when end-of-sequence token is generated
            no_repeat_ngram_size=2  # Prevent repetition of 2-grams
        )
        gen_time = time.time() - gen_start

        # Decoding time
        decode_start = time.time()
        generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
        decode_time = time.time() - decode_start

        # Total time for this run
        total_time += time.time() - start_time
        generated_outputs.append(generated)

        if num_runs > 1:
            print(f"\nRun {run + 1}:")
        print(f"Tokenization time: {token_time*1000:.2f} ms")
        print(f"Generation time: {gen_time*1000:.2f} ms")
        print(f"Decoding time: {decode_time*1000:.2f} ms")
        print(f"Total time: {(time.time() - start_time)*1000:.2f} ms")

    if num_runs > 1:
        avg_time = total_time / num_runs
        print(f"\nAverage time over {num_runs} runs: {avg_time*1000:.2f} ms")

    return generated_outputs[0] if num_runs == 1 else generated_outputs

In [ ]:
# Test the original model
prompt = "Paris is the capital of"
generated = get_output(prompt, num_runs=2)
print(f"Generated text: {generated}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Run 1:
Tokenization time: 1.84 ms
Generation time: 2701.53 ms
Decoding time: 0.27 ms
Total time: 2703.73 ms

Run 2:
Tokenization time: 0.55 ms
Generation time: 1657.01 ms
Decoding time: 0.21 ms
Total time: 1657.85 ms

Average time over 2 runs: 2180.71 ms
Generated text: ['Paris is the capital of France. It is located in the north-central part of the country, on the river Seine. The city has a population of over 2 million people, making it the largest city in France and the second-largest city', 'Paris is the capital of France. It is located in the north-central part of the country, on the river Seine. The city has a population of over 2 million people, making it the largest city in France and the second-largest city']


The text generation of the original model, as expected, works perfectly and returns a correct and meaningful sentence.

In [ ]:
model.to("cpu")               # actual data moves ↙
torch.cuda.empty_cache()      # allocator drops cached blocks

# Pruning the Model.

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from copy import deepcopy

This function `measure_unpruned_layer_importances` is designed to calculate importance scores for the attention layers in a model.

The basic idea is: if a layer's output is very similar to its input, it might not be doing much important work and could be a candidate for pruning. To check the difference I'm using the cosine similarity.

In [ ]:
def measure_unpruned_layer_importances(pruned_model, tokenizer, input_text):
    """
    Measures and returns importance scores for all unpruned (non-bypassed) layers.
    """
    # PREPARATION
    """
    set the model to evaluation mode to ensure that no gradients
    are computed during the forward pass.
    """
    pruned_model.eval()
    device = next(pruned_model.parameters()).device

    """
    The provided input text (input_text) is tokenized into tensors
    suitable for processing by the model.
    """
    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    """This will hold tuples of (layer_idx, importance_score)"""
    importance_scores = []

    # IDENTIFY UNPRUNED LAYERS & CREATING HOOKS
    """
    We'll register hooks for only layers that are NOT in drop_attn_list
    The list of attention layers that have already been pruned,
    is stored in a variable in the model's config: pruned_model.config.drop_attn_list.
    """
    unpruned_layer_indices = [
        idx for idx in range(len(pruned_model.model.layers))
        if idx not in pruned_model.config.drop_attn_list
    ]

    """
    Temporary storage for each layer's input/output
    We'll store them by layer index
    """
    layer_inputs = {}
    layer_outputs = {}

    """
    Create 2 hooks to capture the input and the output of the layers.
    These hooks store the inputs and outputs in dictionaries
    (layer_inputs and layer_outputs) for later analysis
    """
    #Allows capture the input to the query projection (q_proj)
    def q_proj_input_hook(layer_idx):
        def _hook(module, module_input):
            # module_input can be a tuple depending on PyTorch version
            inp = module_input[0] if isinstance(module_input, tuple) else module_input
            layer_inputs[layer_idx] = inp.detach().clone()
        return _hook

    # Allows capture the output from the output projection (o_proj)
    def o_proj_output_hook(layer_idx):
        def _hook(module, module_input, module_output):
            out = module_output[0] if isinstance(module_output, tuple) else module_output
            layer_outputs[layer_idx] = out.detach().clone()
        return _hook

    # Register hooks for each unpruned layer
    handles = []
    for idx in unpruned_layer_indices:
        layer = pruned_model.model.layers[idx]
        handles.append(layer.self_attn.q_proj.register_forward_pre_hook(q_proj_input_hook(idx)))
        handles.append(layer.self_attn.o_proj.register_forward_hook(o_proj_output_hook(idx)))

    # FORWARD PASS
    """
    Single forward pass (no gradient needed)
    A single forward pass is performed on the input text.
    During this pass, the hooks capture the inputs and outputs of the unpruned layers.
    This step is done with torch.no_grad(),
    ensuring no gradients are calculated, which saves memory and computation.
    """
    with torch.no_grad():
        _ = pruned_model(**inputs)

    """
    The hooks are removed after the forward pass
    to avoid memory leaks or interference with subsequent operations.
    """
    for h in handles:
        h.remove()


    #COMPUTE IMPORTANCE SCORES
    """
    For each unpruned layer, the inputs and outputs are flattened into vectors for comparison.

    Cosine Similarity: The similarity between the input and output vectors is
    computed using cosine similarity. Layers with outputs that are very similar
    to their inputs likely contribute less to the model’s overall computation.

    Importance Score: The importance score for each layer is calculated as 1−similarity
    A higher score indicates that the layer transforms its input significantly
    and is therefore more important to the model's function.
    """
    for idx in unpruned_layer_indices:
        if idx in layer_inputs and idx in layer_outputs:
            inp = layer_inputs[idx]
            out = layer_outputs[idx]

            inp_flat = inp.view(inp.size(0), -1)
            out_flat = out.view(out.size(0), -1)

            similarity = F.cosine_similarity(inp_flat, out_flat, dim=1).mean().item()
            importance_score = 1 - similarity
            importance_scores.append((idx, importance_score))

            print(f"Layer {idx} importance score: {importance_score:.4f}")

    """A list of tuples is returned, where each tuple contains the layer index
    and its calculated importance score."""
    return importance_scores


The function `bypass_single_layer` is used to disable the attention mechanism of a specific layer in the model without permanently removing or modifying the layer.

This is achieved by dynamically overriding the layer’s forward method to bypass its attention computation.

As the attention layers are grouped with the MLP Layers we can just remove an attention layer without removing the associated MLP layer. But we can bypass the layer.

The bypassed layer skips computationally expensive attention operations, reducing inference time and memory usage.


In [ ]:
def bypass_single_layer(pruned_model, layer_idx):
    """
    Modifies the specified layer's forward method so that attention is bypassed.
    """
    layer = pruned_model.model.layers[layer_idx]

    # get the list once, while we still have access to the full config
    skip = pruned_model.config.drop_attn_list
    layer.self_attn.layer_idx = layer_idx

    # Store the original forward.
    if not hasattr(layer.self_attn, '_original_forward'):
        layer.self_attn._original_forward = layer.self_attn.forward

    # Set a simple bypass flag directly on the attention layer
    layer.self_attn._should_bypass = True

    # A new forward that checks the bypass flag
    def new_attention_forward(attn, hidden_states, attention_mask=None, position_ids=None,
                    past_key_value=None, output_attentions=False, use_cache=False,
                    **kwargs):
        if attn.layer_idx in skip:
            # short-circuit the pruned layer
            return (hidden_states, None) if use_cache else (hidden_states, None)
        return attn._orig_forward(hidden_states, attention_mask, position_ids,
                                  past_key_value, output_attentions, use_cache,
                                  **kwargs)
    #set new forward method
    layer.self_attn.forward = new_attention_forward.__get__(layer.self_attn,
                                                  type(layer.self_attn))



In [ ]:
def one_shot_pruning_inplace(model, tokenizer, input_text, num_layers_to_prune):
    """
    Performs pruning on the original model without creating a copy.
    """
    # Save original device (should be CPU in your case)
    device = next(model.parameters()).device

    # Set up pruning list
    if not hasattr(model.config, 'drop_attn_list'):
        model.config.drop_attn_list = []

    # Measure importance
    scores = measure_unpruned_layer_importances(model, tokenizer, input_text)

    if len(scores) < num_layers_to_prune:
        raise ValueError("Requested more layers to prune than exist")

    # Sort and select layers to bypass
    scores.sort(key=lambda x: x[1])  # ascending
    layers_to_bypass = [idx for idx, _ in scores[:num_layers_to_prune]]  # Fixed syntax error

    # Bypass selected layers
    for idx in layers_to_bypass:
        model.config.drop_attn_list.append(idx)
        bypass_single_layer(model, idx)
        print(f"Bypassing layer {idx} with importance score {dict(scores)[idx]:.4f}")

    print(f"Bypassed layers: {sorted(model.config.drop_attn_list)}")

    return model

## Execute Pruning.

**Disclaimer**

I'm using a single illustrative prompt so that the code path is easy to follow. In any research or production setting you must feed hundreds or thousands of diverse prompts before deciding which layers to deactivate

In [ ]:
pruned_model = one_shot_pruning_inplace(
      model,
      tokenizer,
       "Hi I'm a sample text, used to calculate the cosine difference between input and output.",
      num_layers_to_prune=4
)

Layer 0 importance score: 1.0711
Layer 1 importance score: 0.9268
Layer 2 importance score: 0.9592
Layer 3 importance score: 0.9586
Layer 4 importance score: 0.9556
Layer 5 importance score: 1.0017
Layer 6 importance score: 1.0273
Layer 7 importance score: 1.0322
Layer 8 importance score: 1.1082
Layer 9 importance score: 1.1019
Layer 10 importance score: 1.0403
Layer 11 importance score: 0.9950
Layer 12 importance score: 1.0824
Layer 13 importance score: 1.0197
Layer 14 importance score: 0.9663
Layer 15 importance score: 0.9675
Layer 16 importance score: 0.9030
Layer 17 importance score: 0.9355
Layer 18 importance score: 1.0156
Layer 19 importance score: 0.7300
Layer 20 importance score: 0.8689
Layer 21 importance score: 0.9351
Layer 22 importance score: 0.8706
Layer 23 importance score: 0.7950
Layer 24 importance score: 0.8481
Layer 25 importance score: 0.8776
Layer 26 importance score: 0.7842
Layer 27 importance score: 0.9855
Bypassing layer 19 with importance score 0.7300
Bypassing 

# Test Pruned Models


Now, let's test the pruned model, which is a Llama-3.2-3B model where I have marked 4 Attention layers to be bypassed.

In [ ]:
# Test the pruned model
pruned_model = pruned_model.to(device) #Move the model to GPU again.
generated = get_output(prompt, pruned_model, num_runs=2)
print(f"Generated text: {generated}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Run 1:
Tokenization time: 0.44 ms
Generation time: 1508.45 ms
Decoding time: 0.23 ms
Total time: 1509.20 ms

Run 2:
Tokenization time: 0.49 ms
Generation time: 1506.98 ms
Decoding time: 0.19 ms
Total time: 1507.74 ms

Average time over 2 runs: 1508.39 ms
Generated text: ['Paris is the capital of France and also its largest city. It is also one of the most visited tourist destination worldwide with millions of tourists visiting every year. There are many things to do in Paris including sightseeing tours, shopping malls, museums etc', 'Paris is the capital of France and also its largest city. It is also one of the most visited tourist destination worldwide with millions of tourists visiting every year. There are many things to do in Paris including sightseeing tours, shopping malls, museums etc']



The execution of this second model is slightly faster than that of the base model, and the generated text is fairly accurate, although some repetition can be noticed towards the end of the sentence.

# Store the Model.


In [ ]:
new_model_name = 'attnprun-llama-3.2-3B'
output_dir = './'+new_model_name
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

pruned_model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
#new_config.save_pretrained(output_dir)
print(f"Pruned model saved to {output_dir}")

Pruned model saved to ./attnprun-llama-3.2-3B


In [ ]:
# 2. Check that config contains layers to skip
from transformers import AutoConfig
config = AutoConfig.from_pretrained(output_dir)

if hasattr(config, "drop_attn_list"):
    print(f"drop_attn_list stored: {config.drop_attn_list}")
else:
    print("drop_attn_list isn't present.")


drop_attn_list stored: [19, 26, 23, 24]


## Upload to Hugging Face.

El proceso de subida de este modelo a Hugging es ligeramente más complejo por que se debe almacenar no tan solo el modelo en si, sino tambien el código de la función _bypass_single_layer. Que como recordarás es la función que se encarga de decidir cuando ejecutar o simplemente bypasear una capa de atención.  

In [ ]:
from huggingface_hub import HfApi, upload_folder, whoami, create_repo

In [ ]:
# Step 1: Get your HF username from the current token
username = whoami()["name"]  # Returns a dict like {'name': 'your_username', 'email': ...}
username

'oopere'

In [ ]:
# Step 2: Define repo name
repo_id = f"{username}/{new_model_name}"

In [ ]:
# Step 3: Define path to your model
output_dir = "./"+new_model_name


The function must be saved in a .py file, but since this notebook runs on Colab, I’ve decided the best approach is to create a cell that generates the file to be uploaded.

The file contains the custom class PrunedLlamaForCausalLM, which extends Hugging Face’s LlamaForCausalLM.

This custom class calls the base constructor, ensuring that the model's configuration file includes the drop_attn_list, which specifies the layers that should be skipped.

The forward function is modified only for the layers that need to be skipped; the rest continue executing their standard forward function.


In [ ]:
custom_model_code = '''
from transformers.models.llama.modeling_llama import LlamaForCausalLM

class PrunedLlamaForCausalLM(LlamaForCausalLM):
    def __init__(self, config):
        super().__init__(config)
        if not hasattr(config, "drop_attn_list"):
            config.drop_attn_list = []

        for idx in config.drop_attn_list:
            self._bypass_single_layer(idx)

    def _bypass_single_layer(pruned_model, layer_idx):
        """
        Modifies the specified layer's forward method so that attention is bypassed.
        """
        layer = pruned_model.model.layers[layer_idx]

        # get the list once, while we still have access to the full config
        skip = pruned_model.config.drop_attn_list
        layer.self_attn.layer_idx = layer_idx

        # Store the original forward.
        if not hasattr(layer.self_attn, '_original_forward'):
            layer.self_attn._original_forward = layer.self_attn.forward

        # Set a simple bypass flag directly on the attention layer
        layer.self_attn._should_bypass = True

        # A new forward that checks the bypass flag
        def new_attention_forward(attn, hidden_states, attention_mask=None, position_ids=None,
                        past_key_value=None, output_attentions=False, use_cache=False,
                        **kwargs):
            if attn.layer_idx in skip:
                # short-circuit the pruned layer
                return (hidden_states, None) if use_cache else (hidden_states, None)
            return attn._orig_forward(hidden_states, attention_mask, position_ids,
                                      past_key_value, output_attentions, use_cache,
                                      **kwargs)
        #set new forward method
        layer.self_attn.forward = new_attention_forward.__get__(layer.self_attn,
                                                      type(layer.self_attn))

'''

# Define path and write the file
os.makedirs(output_dir, exist_ok=True)
with open(os.path.join(output_dir, "modeling_attnprun_llama.py"), "w") as f:
    f.write(custom_model_code.strip())

print("Custom model script modeling_attnprun_llama.py created successfully.")


Custom model script modeling_attnprun_llama.py created successfully.


Now the model's configuration file is updated by adding the `auto_map` field, which tells the Transformers library which class to use to construct the model: `modeling_attnprun_llama.PrunedLlamaForCausalLM.`


In [ ]:
import json
import os

# Path to the config file
config_path = os.path.join(output_dir, "config.json")

# Load the existing config
with open(config_path, "r") as f:
    config = json.load(f)

# Add or update the auto_map section
config["auto_map"] = {
    "AutoModelForCausalLM": "modeling_attnprun_llama.PrunedLlamaForCausalLM"
}

# Optional: ensure the architecture field is aligned
config["architectures"] = ["PrunedLlamaForCausalLM"]

# Save the updated config
with open(config_path, "w") as f:
    json.dump(config, f, indent=2)

print("config.json updated with auto_map and architecture.")

config.json updated with auto_map and architecture.


Time to upload the folder containing the weights, the config file and the new function to HF.

In [ ]:
create_repo(repo_id=repo_id, repo_type="model", exist_ok=True)

RepoUrl('https://huggingface.co/oopere/attnprun-llama-3.2-3B', endpoint='https://huggingface.co', repo_type='model', repo_id='oopere/attnprun-llama-3.2-3B')

In [ ]:
# Step 4: Upload the folder to the Hub
upload_folder(
    folder_path=output_dir,
    path_in_repo="",  # Upload everything to root
    repo_id=repo_id,
    repo_type="model"
)

print(f"Model uploaded successfully to https://huggingface.co/{repo_id}")

No files have been modified since last commit. Skipping to prevent empty commit.


Model uploaded successfully to https://huggingface.co/oopere/attnprun-llama-3.2-3B


## Download model from Hugging Face.

In [ ]:
import gc
del pruned_model
del tokenizer
del model

# 2. Libera la caché de la GPU
torch.cuda.empty_cache()
torch.cuda.ipc_collect()  # Opcional, ayuda en Colab

# 3. Forza recolección de basura en Python
gc.collect()

573

In [ ]:
!huggingface-cli cache purge --yes

usage: huggingface-cli <command> [<args>]
huggingface-cli: error: argument {download,upload,repo-files,env,login,whoami,logout,auth,repo,lfs-enable-largefiles,lfs-multipart-upload,scan-cache,delete-cache,tag,version,upload-large-folder}: invalid choice: 'cache' (choose from 'download', 'upload', 'repo-files', 'env', 'login', 'whoami', 'logout', 'auth', 'repo', 'lfs-enable-largefiles', 'lfs-multipart-upload', 'scan-cache', 'delete-cache', 'tag', 'version', 'upload-large-folder')


The model is downloaded normally from Hugging Face, but you must remember to set `trust_remote_code=True` since the model includes the custom code you previously created and uploaded.


In [ ]:
model_hf = AutoModelForCausalLM.from_pretrained(
    repo_id,
    trust_remote_code=True
)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(repo_id)

In [ ]:
model_hf = model_hf.to(device) #Move the model to GPU again.
generated = get_output(prompt, model_hf, num_runs=2)
print(f"Generated text: {generated}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Run 1:
Tokenization time: 0.74 ms
Generation time: 1530.55 ms
Decoding time: 0.20 ms
Total time: 1531.58 ms

Run 2:
Tokenization time: 0.53 ms
Generation time: 1535.94 ms
Decoding time: 0.21 ms
Total time: 1536.77 ms

Average time over 2 runs: 1534.09 ms
Generated text: ['Paris is the capital of France and also its largest city. It is also one of the most visited tourist destination worldwide with millions of tourists visiting every year. There are many things to do in Paris including sightseeing tours, shopping malls, museums etc', 'Paris is the capital of France and also its largest city. It is also one of the most visited tourist destination worldwide with millions of tourists visiting every year. There are many things to do in Paris including sightseeing tours, shopping malls, museums etc']


# Conclusion.
Based on the findings in the paper and the results obtained, I believe this type of pruning may work better with larger models where attention layers tend to have redundancy.

Since this type of pruning does not alter the model's structure, it does not result in a reduction in its size or the memory required to load it. The main advantage of using this pruning approach is the reduction of computational load during inference, leading to a more efficient model with faster responses and lower resource consumption.

Unlike the original paper, which describes "removing" selected attention layers but provides limited implementation details, this implementation takes a transparent functional approach by explicitly overriding the `forward` method only in the specified layers. As a result, the model retains its full architecture and parameter set, but selectively skips computations at runtime. This makes the method reversible, modular, and fully compatible with the Hugging Face ecosystem using `trust_remote_code=True`. While both approaches achieve similar computational savings, this one emphasizes clarity, portability, and practical integration.


# Authors Note.

In addition to creating content like this notebook and offering it under the MIT license, I have also contributed to repositories such as those of Hugging Face and Google Gemini.

I am especially proud of my book: [Large Language Models: Apply and Implement Strategies for Large Language Models (Apress)(https://amzn.to/3DSepLb).

You can find it on both [Amazon](https://amzn.to/3DSepLb) and [Springer](https://link.springer.com/book/10.1007/979-8-8688-0515-8), where they often have good deals on the purchase price.

If you take a look and end up purchasing it, keep in mind that you can reach out with any questions via the Discussions section of this same repository or on any of my social media channels. I’ll do my best to respond as quickly as possible.